In [1]:
import os
from pytesseract import Output
import re
import cv2 #opencv-python-headless             4.5.4.58/ ahora opencv-python                      4.5.5.62
import pytesseract
#import easyocr
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score
import pandas as pd
from scipy.stats import norm
from scipy.stats import rv_histogram
import time

In [2]:
pytesseract.pytesseract.tesseract_cmd = r'D:\Program Files\Tesseract-OCR\tesseract.exe'
custom_config = r'--oem 3 --psm 1 -l spa'

In [3]:
# get grayscale image
def get_grayscale(image):
    return cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

# noise removal
def remove_noise(image):
    return cv2.GaussianBlur(image, (7, 7), 0)
 
#thresholding
def thresholding(img):
    ret2,th2 = cv2.threshold(img,0,255,cv2.THRESH_BINARY+cv2.THRESH_OTSU)
    return th2

#dilation
def dilate(image):
    kernel = np.ones((5,5),np.uint8)
    return cv2.dilate(image, kernel, iterations = 2)#1
    
#erosion
def erode(image):
    kernel = np.ones((5,5),np.uint8)
    return cv2.erode(image, kernel, iterations = 3)#3

#opening - erosion followed by dilation
def opening(image):
    kernel = np.ones((5,5),np.uint8)
    return cv2.morphologyEx(image, cv2.MORPH_OPEN, kernel)

#C
def canny(image):
    return cv2.Canny(image, 100, 200)

#Rotación
def getSkewAngle(cvImage):
    newImage = cvImage.copy()
    gray = cv2.cvtColor(newImage, cv2.COLOR_BGR2GRAY)
    blur = cv2.GaussianBlur(gray, (9, 9), 0)
    thresh = cv2.threshold(blur, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)[1]

    # Apply dilate to merge text into meaningful lines/paragraphs.
    # Use larger kernel on X axis to merge characters into single line, cancelling out any spaces.
    # But use smaller kernel on Y axis to separate between different blocks of text
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (30, 5))
    dilate = cv2.dilate(thresh, kernel, iterations=5)

    # Find all contours
    contours, hierarchy = cv2.findContours(dilate, cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)
    contours = sorted(contours, key = cv2.contourArea, reverse = True)

    # Find largest contour and surround in min area box
    largestContour = contours[0]
    minAreaRect = cv2.minAreaRect(largestContour)

    # Determine the angle. Convert it to the value that was originally used to obtain skewed image
    angle = minAreaRect[-1]
    if angle < -45:
        angle = 90. + angle
    print(angle)
    return -1 * angle + 90
# Rotar alrededor del centro
def rotateImage(cvImage, angle):
    newImage = cvImage.copy()
    (h, w) = newImage.shape[:2]
    center = (w // 2, h // 2)
    M = cv2.getRotationMatrix2D(center, angle, 1.0)
    newImage = cv2.warpAffine(newImage, M, (w, h), flags=cv2.INTER_CUBIC, borderMode=cv2.BORDER_REPLICATE)
    return newImage

# Juntar
def deskew(cvImage):
    angle = getSkewAngle(cvImage)
    return rotateImage(cvImage, -1.0 * angle)

#template matching
def match_template(image, template):
    return cv2.matchTemplate(image, template, cv2.TM_CCOEFF_NORMED)

#Returns text in image
def imagetotext(imagepath): #imagepath=str
    img = cv2.imread(imagepath)
    text = pytesseract.image_to_string(img, config=custom_config)
    return text
    
def takepicture():
    webcam = cv2.VideoCapture(0)
    check, frame = webcam.read()
    cv2.imwrite(filename=r'saved_img.jpg', img=frame)
    webcam.release()

def tts(text):
    engine.say(text);
    engine.runAndWait()

def remove_newlines(text):
        return text.replace("\n"," ") #para quitar las nuevas líneas y que lea con más fluidez
    
def leertxt(archivotxt):
    content = open(archivotxt, "r", encoding='utf-8').read()
    lecturatxt = re.split('\s|\n',content)
    try:
        while True:
            lecturatxt.remove('')
    except ValueError:
        pass
    return lecturatxt

def listafiltrada(archivoimagen):
    img = cv2.imread(archivoimagen)
    text = pytesseract.image_to_data(img, config=custom_config, output_type=Output.DATAFRAME).dropna()
    #text['text'] = text['text'].lower()
    lista = text["text"].tolist()
    try:
        while True:
            lista.remove('')
    except ValueError:
        pass
    try:
        while True:
            lista.remove(' ')
    except ValueError:
        pass
    try:
        while True:
            lista.remove('  ')
    except ValueError:
        pass
    if lista[-1].isnumeric() == True:
        lista.pop(-1)
    filterlist = []
    skip = []
    for i in range(len(lista)): #para juntar última palabra de línea con siguiente
        if i in skip:
            continue
        if lista[i].find("-")!=-1:
            if i+1>=len(lista): #por si es la última palabra de la lista
                break
            else:
                filterlist.append(lista[i].replace('-', '')+lista[i+1]) 
                skip.append(i+1)
        else:
            filterlist.append(lista[i])
    try:
        while True:
            filterlist.remove('')
    except ValueError:
        pass
    try:
        while True:
            filterlist.remove(' ')
    except ValueError:
        pass
    try:
        while True:
            filterlist.remove('  ')
    except ValueError:
        pass
    return filterlist

def accuracyAlonso(lista1,lista2):
    p = 0
    j = 0
    skip = []
    minimo = min([len(lista1),len(lista2)])
    i = 0
    while i < minimo:
        if i in skip:
            break
        if lista1[i]==lista2[i]:
            p += 1
        elif lista1[i]==lista2[i+1]:
            p += 1
            lista2.pop(i)
            j += 1
            skip.append(minimo-j-1)
        elif lista1[i+1]==lista2[i]:
            p += 1
            lista1.pop(i)
            j += 1
            skip.append(minimo-j-1)            
        i += 1
    return p/minimo

def locateRegion(img):
    #gray
    gray = get_grayscale(img)
    blurred = remove_noise(gray)
    dilated = dilate(blurred)
    eroded = erode(dilated)
    thresh = cv2.threshold(eroded, 0, 255, cv2.THRESH_BINARY | cv2.THRESH_OTSU)[1]
    contours, hierarchy = cv2.findContours(image=thresh, mode=cv2.RETR_TREE, method=cv2.CHAIN_APPROX_NONE)
    image_copy = img.copy()
    cv2.drawContours(image=image_copy, contours=contours, contourIdx=-1, color=(0, 255, 0), thickness=2, lineType=cv2.LINE_AA)
    all_areas= []
    for cnt in contours:
        area= cv2.contourArea(cnt)
        all_areas.append(area)
    sorted_contours= sorted(contours, key=cv2.contourArea, reverse= True)
    mask = np.zeros_like(image_copy) # Queremos que nuestra región de interés sea blanca, y si no, negra
    cv2.drawContours(mask, sorted_contours, 0, (255,255,255), -1) # Hacemos que la región de interés sea blanca
    out = np.zeros_like(image_copy)
    out.fill(255)
    out[mask == 255] = img[mask == 255] # Extraemos el objeto y lo ponemos sobre nuestra "máscara"
    return out

In [5]:
directory = 'C:/Users/alons/Desktop/Universidad/Universidad21.22/TFG/Vision/WA'
t = []
for filename in os.listdir(directory):
    if filename.endswith(".jpeg"):
        start = time.perf_counter()
        takepicture()
        img = cv2.imread(filename, 0)
        img = thresholding(img)
        cv2.imwrite('prueba.jpeg', img)
        img = cv2.imread('prueba.jpeg')
        img = locateRegion(img)
        img = deskew(img)
        cv2.imwrite('prueba.jpeg', img)
        text = imagetotext('prueba.jpeg')
        end = time.perf_counter()
        t.append(round(end-start, 2))
print(f'Finished in {round(end-start, 2)} second(s)')

90.0
90.0
90.0
90.0
Finished in 3.45 second(s)


In [8]:
data = np.load('tiemposrpi.npy')

In [9]:
print(data)

[22.82 14.13 16.74 15.16 16.04 12.33 16.74 16.74 13.58 13.76 14.74 19.72
 14.69 17.07 16.16 23.74 16.63 16.46 19.71 14.79 15.76 19.06 14.53 17.68
 16.92 18.22 13.37 20.52 18.57 14.93 13.7  14.42 14.17 12.89 14.28 15.74
 17.44 14.93 12.24 16.76 15.85 20.96 19.77 14.96 14.53 13.81 21.   15.61
 12.68]
